# Úvod do DEAP (Distributed Evolutionary Algorithms in Python)

instalace v terminálu pomocí 

conda install -c conda-forge deap 

In [ ]:
import numpy as np

import random

from deap import base, creator, tools, algorithms



## Vytváření nových typů
---

### Fitness
params: název, objekt fitness, -1.0 minimalizace / 1.0 maximalizace

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

### Individua
params: název, třída datové struktury, fitness dle existující zaregistrované

budeme pracovat s listy (jde i numpy pole ale pozor na views!)

existuje celá řada typů: permutace, stromy, atd.

In [ ]:
creator.create("Individual", list, fitness=creator.FitnessMax)

In [ ]:
toolbox = base.Toolbox()

toolbox.register("attr_float", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=IND_SIZE)
#toolbox.register("individual", initind, IND_LEN)  # název, funkce, poporade parametry krmici funkci

In [ ]:
ind1 = toolbox.individual()
print(ind1)
ind1[1]

In [ ]:
ind1.fitness.values


### Populace

In [ ]:
toolbox.register("population", tools.initRepeat, list, toolbox.individual) 
# init repeat opakuje dle parametru za ni, tedy dela list individui dle zakladni inicializace individua


In [ ]:
pop = toolbox.population(n=10)
print(pop)

## Fitness
---
Tuto funkci si musíme napsat sami !!!

In [ ]:
def evaluate(individual):
    return sum(individual),    # !!!! vracíme n-tici, proto ta čárka

In [ ]:
toolbox.register("evaluate", evaluate)

## Operátory
---

sada operátorů lze nalézt v [dokumentaci](https://deap.readthedocs.io/en/master/api/tools.html#module-deap.tools)

In [ ]:
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=1, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

## Jednoduchý algoritmus
---

In [ ]:
# hlavní parametry vystrčené kvůli experimentování

NGEN = 50            # počet generací
CXPB = 0.7           # pravděpodobnost crossoveru na páru
MUTPB = 0.2         # pravděpodobnost mutace jedince

finalpop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN)


## Statistika
---

In [ ]:
import numpy as np

s = tools.Statistics(key=lambda ind: ind.fitness.values)
s.register("mean", np.mean)
s.register("max", np.max)


hof = tools.HallOfFame(1)  # pamatuje si 1 nejlepšího jedince za historii evoluce (i když zanikne)

pop = toolbox.population(n=10)


finalpop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, stats=s, halloffame=hof)

In [ ]:
mean, maximum = logbook.select("mean", "max")


print(hof)

## Kreslení statistiky
---

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

ax.plot(range(NGEN+1), mean, label="mean")     # 0.tá generace zvlášť
ax.plot(range(NGEN+1), maximum, label="max")

ax.legend()

## Aplikace na barvení grafu
---

### Načtení dat

In [ ]:
import numpy as np
import networkx as nx



# funkce pro nacitani grafu z Dimacs formatu
def readdimacs(filename):

    file = open(filename, 'r')
    lines = file.readlines()
    
    Gd = nx.Graph()

    for line in lines:
        if line[0] == "e":
            vs = [int(s) for s in line.split() if s.isdigit()]
            Gd.add_edge(vs[0]-1, vs[1]-1)   # dimacs cisluje vrcholy od 1
    return Gd

Gd = nx.Graph()
Gd = readdimacs('dsjc125.1.txt') 




### Definice GA

In [ ]:

COLNUM = 10 # počet barev

N = Gd.number_of_nodes()

creator.create("FitnessMax", base.Fitness, weights=(-1.0,))  # minimalizujeme počet kolizí

creator.create("Individual", list, fitness=creator.FitnessMax)


toolbox = base.Toolbox()

toolbox.register("attr_float", random.randint, 0, COLNUM)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attr_float, n=N)


toolbox.register("population", tools.initRepeat, list, toolbox.individual) 


# vrací celkový počet kolizí
def evaluate(individual):
    
    collisions = 0
    
    for i in range(N-1):
        for j in range(i, N):
            if Gd.has_edge(i, j):
                if individual[i] == individual[j]:
                    collisions += 1
    return collisions, # !!!! vracíme n-tici, proto ta čárka
   

toolbox.register("evaluate", evaluate)


toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutUniformInt, low=0, up=COLNUM, indpb=0.01)
toolbox.register("select", tools.selTournament, tournsize=2)



NGEN = 150          # počet generací
CXPB = 0.5           # pravděpodobnost crossoveru na páru
MUTPB = 0.7         # pravděpodobnost mutace



s = tools.Statistics(key=lambda ind: ind.fitness.values)
s.register("mean", np.mean)
s.register("min", np.min)

pop = toolbox.population(n=100)


finalpop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, stats=s)

### Vykreslení fitness

In [ ]:
mean, minimum = logbook.select("mean", "min")

fig, ax = plt.subplots()

ax.plot(range(NGEN+1), mean, label="mean")     # 0.tá generace zvlášť
ax.plot(range(NGEN+1), minimum, label="min")
ax.legend()